In [46]:
from langchain import PromptTemplate
from langchain.chains import retrieval_qa
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain_community.vectorstores import Pinecone as langchainPinecone

In [68]:
# Step 1: Import classes
from pinecone import Pinecone, ServerlessSpec

# Step 2:  Pinecone API key
PINECONE_API_KEY = "pcsk_bkA7R_QrV1CyjrzUqiiVJrbb4cwySPnt6PsVgn2QfHq6ys8eWyAy3gFK5vGgtYLRCsT3x"
index_name = "medical-chatbot"

#Step 4: Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Step 5: Connect to existing index
index = pc.Index(index_name)
print(f" Connected to existing index '{index_name}'")



 Connected to existing index 'medical-chatbot'


In [70]:
# Extract data from PDF
def load_pdf(data):
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [71]:
extracted_data = load_pdf("data/")

In [72]:
# Create text chunks 
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [73]:
text_chunks = text_split(extracted_data)
print("Length of my chunks:",len(text_chunks))

Length of my chunks: 39994


In [74]:
text_chunks[0]

Document(metadata={'source': 'data\\medical_book.pdf', 'page': 1}, page_content='The GALE\nENCYCLOPEDIA of\nMEDICINE\nTHIRD EDITION')

In [75]:
# Download embedding model 
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [76]:
embeddings = download_hugging_face_embeddings()

In [22]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [77]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [83]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY  # ✅ This line is the fix

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

# Connect to index
index = pc.Index(index_name)

limited_chunks = text_chunks[:10000]
# Upload using Langchain
docsearch = PineconeVectorStore.from_texts(
    texts=[t.page_content for t in limited_chunks],
    embedding=embeddings,
    index_name=index_name,
    namespace="default"
)

In [ ]:
# If we already have an index we can load it like this
docsearch = PineconeVectorStore.from_existing_index(index_name,embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query,k=3)

print("Result",docs)

Result []


In [90]:
Prompt_template = """Use the following pieces of information to answer the 
user question . if you dont know the answer, just say that you dont know , dont try to makeup the answer

context:{context}
question:{question}

Only return the helpful answer below and nothing else.
Helpful answer"""

In [91]:
PROMPT = PromptTemplate(template=Prompt_template,input_variables=["context","question"])
chain_type_kwargs = {"prompt":PROMPT}

In [118]:
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

# Load LLM
llm = CTransformers(
    model="E:/5)End-To-End-Medical-Chatbot/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        "max_new_tokens": 512,
        "temperature": 0.8
    }
)

# Create RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({"query":user_input})
    print("Response:",result["result"])